In [1]:
import pandas as pd
from pandas.core.frame import DataFrame

from pathlib import Path
from collections import Counter
import unicodedata


In [2]:
# Define some helper functions

def csv_loader(path:Path) ->list[DataFrame]:
    """Given a path, extracts and loads every csv recursively and then loads it into pd.df
    """
    files = path.rglob('*.csv')
    return [pd.read_csv(filepath, sep=";", low_memory=False) for filepath in files]


def print_columns(dataframes:list[DataFrame], length=False, unicode=False, repr=False) -> None:
    """Print columns and ocurrences and more
    """
    all_columns = [col for df in dataframes for col in df.columns]

    column_counts = Counter(all_columns)

    print("Column name occurrences:")
    for col, count in column_counts.items():
        print(f"'{col}' -> **{count}**")
        if length:
            print(f"  - Length: {len(col)}")
        if unicode:
            print(f"  - Unicode name: {[unicodedata.name(c) for c in col]}")
        if repr:
            print(f"  - Repr: {repr(col)}")




In [3]:

# The paths and columns to keep of the information we will be working with

column_path_dict = {
    "denuncias": [
        "../data/raw/denuncias/",
        [
            "agno",
            "rbd",
            "den_canal",
            "den_estado",
            "den_fec_creacion",
            "den_fec_termino_denuncia",
            "den_oficina",
            "den_departamento",
            "den_ambito",
            "den_tema",
            "den_subtema",
            "den_sexo",
        ],
    ],
    "ee_info": [
        "../data/raw/info_estab/",
        [
            "agno",
            "rbd",
            "nom_rbd",
            "nom_com_rbd",
            "cod_com_rbd",
            "cod_reg_rbd",
            "cod_depe",
            "cod_depe2",
            "rural_rbd",
            "latitud",
            "longitud",
            "convenio_pie",
            "ori_religiosa",
            "pago_matricula",
            "pago_mensual",
        ],
    ],
    "ee_sep": [
        "../data/raw/estab_sep/",
        [
            "agno",
            "rbd",
            "clasificacion_sep",
            "convenio_sep",
            "n_prio",
            "n_ben",
            "n_prio_ben",
            "n_pref",
            "n_pref_ben",
        ],
    ],
    "ee_matricula": [
        "../data/raw/matricula/",
        [
            "agno",
            "rbd",
            "nom_rbd",
            "mat_hom_1",
            "mat_hom_2",
            "mat_hom_3",
            "mat_hom_4",
            "mat_hom_5",
            "mat_hom_6",
            "mat_hom_7",
            "mat_hom_8",
            "mat_muj_1",
            "mat_muj_2",
            "mat_muj_3",
            "mat_muj_4",
            "mat_muj_5",
            "mat_muj_6",
            "mat_muj_7",
            "mat_muj_8",
            "mat_hom_tot",
            "mat_muj_tot",
            "mat_total",
        ],
    ],
}

## Denuncias

In [4]:
denuncias_dfs = csv_loader(Path(column_path_dict["denuncias"][0]))

print_columns(denuncias_dfs)


Column name occurrences:
'AGNO' -> **11**
'DEN_ID' -> **11**
'DEN_CANAL' -> **11**
'DEN_ESTADO' -> **11**
'DEN_FEC_CREACION' -> **11**
'DEN_MES_CREACION' -> **11**
'DEN_TRIMESTRE_CREACION' -> **11**
'DEN_FEC_TERMINO_DENUNCIA' -> **11**
'DEN_OFICINA' -> **11**
'DEN_DEPARTAMENTO' -> **11**
'DEN_AMBITO' -> **11**
'DEN_TEMA' -> **11**
'DEN_SUBTEMA' -> **11**
'DEN_MRUN' -> **11**
'DEN_AFECTADO' -> **11**
'DEN_SEXO' -> **11**
'DEN_TIPO' -> **11**
'AFEC_MRUN' -> **11**
'RBD' -> **11**
'EE_NOMBRE' -> **11**
'EE_COD_REGION' -> **11**
'EE_COD_PROVINCIA' -> **11**
'EE_COD_COMUNA' -> **11**
'EE_NOM_COMUNA' -> **11**
'EE_COD_DEPE' -> **11**
'EE_DEPE_AGRUP' -> **11**
'AFEC_COD_ENSE2' -> **11**
'DEN_REGION' -> **11**
'AFEC_SEXO' -> **10**
'FIS_ID' -> **9**
'FIS_ID_SEGUIMIENTO' -> **9**
'PA_ID' -> **9**
'DEN_CIBERBULLYING' -> **9**
'RSIE' -> **8**
'AFEC_TIPO' -> **4**
'DEN_ENVIO_RESP' -> **3**
'DEN_GENERO' -> **3**
'AFEC_GENERO' -> **3**
'DEN_TIPO_INSTITUCION' -> **3**
'SOST_RUT' -> **3**
'SOST_MRUN' 

In [5]:
for df in denuncias_dfs:
    df.rename(columns={column:column.casefold().strip() for column in df.columns}, inplace=True)

In [6]:
df_denun = pd.concat(denuncias_dfs, axis=0, ignore_index=True)

In [7]:
columns_to_drop_denun = [item for item in df_denun.columns if item not in column_path_dict["denuncias"][1]]

In [8]:
df_denun.drop(columns=columns_to_drop_denun, inplace=True)

In [9]:
df_denun = df_denun.replace(r'^\s*$', pd.NA, regex=True)

In [10]:
df_denun.head(1)

,agno,den_canal,den_estado,den_fec_creacion,den_fec_termino_denuncia,den_oficina,den_departamento,den_ambito,den_tema,den_subtema,den_sexo,rbd
0,2014,1,1,20140102,20140108,2,1,CONVIVENCIA,MEDIDAS DISCIPLINARIAS,NO RENOVACIÓN O CANCELACIÓN DE MATRÍCULA A PÁR...,2,283


In [11]:
df_denun["den_fec_creacion"] = pd.to_datetime(df_denun["den_fec_creacion"],format='%Y%m%d')
df_denun["den_fec_termino_denuncia"] = pd.to_datetime(df_denun["den_fec_termino_denuncia"],format='%Y%m%d')

In [12]:
df_denun.to_csv("../data/processed/denuncias.csv")

In [13]:
df_denun.to_parquet("../data/processed/denuncias.parquet")

## Establecimientos Info

In [14]:
ee_dfs = csv_loader(Path(column_path_dict["ee_info"][0]))

print_columns(ee_dfs)

Column name occurrences:
'ïagno' -> **3**
'rbd' -> **3**
'dgv_rbd' -> **3**
'nom_rbd' -> **3**
'cod_reg_rbd' -> **3**
'cod_pro_rbd' -> **3**
'cod_com_rbd' -> **3**
'nom_com_rbd' -> **3**
'cod_depe' -> **3**
'cod_depe2' -> **3**
'rural_rbd' -> **3**
'latitud' -> **3**
'longitud' -> **3**
'convenio_pie' -> **3**
'ens_01' -> **3**
'ens_02' -> **3**
'ens_03' -> **3**
'ens_04' -> **3**
'ens_05' -> **3**
'ens_06' -> **3**
'ens_07' -> **3**
'ens_08' -> **3**
'ens_09' -> **3**
'ens_10' -> **1**
'ori_religiosa' -> **3**
'ori_otro_glosa' -> **3**
'pago_matricula' -> **3**
'pago_mensual' -> **3**
'cod_deprov_rbd' -> **2**
'nom_deprov_rbd' -> **2**
'matricula' -> **2**
'estado_estab' -> **2**
'mrun' -> **1**
'rut_sostenedor' -> **1**
'p_juridica' -> **1**
'AGNO' -> **7**
'RBD' -> **7**
'DGV_RBD' -> **7**
'NOM_RBD' -> **7**
'MRUN' -> **7**
'RUT_SOSTENEDOR' -> **7**
'P_JURIDICA' -> **7**
'COD_REG_RBD' -> **7**
'COD_PRO_RBD' -> **7**
'COD_COM_RBD' -> **7**
'NOM_COM_RBD' -> **7**
'COD_DEPROV_RBD' -> *

In [15]:
for df in ee_dfs:
    df.rename(columns={column:column.casefold().strip() for column in df.columns}, inplace=True)
    
for df in ee_dfs:
    df.rename(columns={"\ufeffagno":"agno"}, inplace=True)
    
for df in ee_dfs:
    df.rename(columns={"ïagno":"agno"}, inplace=True)

In [16]:
print_columns(ee_dfs)

Column name occurrences:
'agno' -> **10**
'rbd' -> **10**
'dgv_rbd' -> **10**
'nom_rbd' -> **10**
'cod_reg_rbd' -> **10**
'cod_pro_rbd' -> **10**
'cod_com_rbd' -> **10**
'nom_com_rbd' -> **10**
'cod_depe' -> **10**
'cod_depe2' -> **10**
'rural_rbd' -> **10**
'latitud' -> **10**
'longitud' -> **10**
'convenio_pie' -> **10**
'ens_01' -> **10**
'ens_02' -> **10**
'ens_03' -> **10**
'ens_04' -> **10**
'ens_05' -> **10**
'ens_06' -> **10**
'ens_07' -> **10**
'ens_08' -> **10**
'ens_09' -> **10**
'ens_10' -> **6**
'ori_religiosa' -> **10**
'ori_otro_glosa' -> **10**
'pago_matricula' -> **10**
'pago_mensual' -> **10**
'cod_deprov_rbd' -> **9**
'nom_deprov_rbd' -> **9**
'matricula' -> **9**
'estado_estab' -> **9**
'mrun' -> **8**
'rut_sostenedor' -> **8**
'p_juridica' -> **8**
'nom_reg_rbd_a' -> **5**
'ens_11' -> **5**
'pace' -> **2**
'mat_total' -> **2**
'espe_01' -> **1**
'espe_02' -> **1**
'espe_03' -> **1**
'espe_04' -> **1**
'espe_05' -> **1**
'espe_06' -> **1**
'espe_07' -> **1**
'espe_0

In [17]:
ee_df = pd.concat(ee_dfs, axis=0, ignore_index=True)

In [18]:
columns_to_drop_ee = [item for item in ee_df.columns if item not in column_path_dict["ee_info"][1]]

In [19]:
columns_to_drop_ee

['dgv_rbd',
 'cod_pro_rbd',
 'ens_01',
 'ens_02',
 'ens_03',
 'ens_04',
 'ens_05',
 'ens_06',
 'ens_07',
 'ens_08',
 'ens_09',
 'ens_10',
 'ori_otro_glosa',
 'cod_deprov_rbd',
 'nom_deprov_rbd',
 'matricula',
 'estado_estab',
 'mrun',
 'rut_sostenedor',
 'p_juridica',
 'nom_reg_rbd_a',
 'ens_11',
 'pace',
 'mat_total',
 'espe_01',
 'espe_02',
 'espe_03',
 'espe_04',
 'espe_05',
 'espe_06',
 'espe_07',
 'espe_08',
 'espe_09',
 'espe_10',
 'espe_11']

In [20]:
ee_df.drop(columns=columns_to_drop_ee, inplace=True)

In [21]:
ee_df = ee_df.replace(r'^\s*$', pd.NA, regex=True)

In [22]:
import locale

# Set the locale to a comma-decimal locale (e.g., German)
locale.setlocale(locale.LC_ALL, 'en_US.UTF-8')

def convert_to_float(value):
    if pd.isna(value):
        return pd.NA
    if isinstance(value, (float, int)):
        return float(value)
    if isinstance(value, str):
        return float(value.replace(',', '.'))
        # try:
        #     return locale.atof(value)
        # except ValueError:
        #     try:
        #         # Fallback to period as decimal separator
        #         return float(value.replace('.','').replace(',', '.'))
        #     except ValueError:
        #         return pd.NA
    return pd.NA


In [23]:
ee_df.head(1)

,agno,rbd,nom_rbd,cod_reg_rbd,cod_com_rbd,nom_com_rbd,cod_depe,cod_depe2,rural_rbd,latitud,longitud,convenio_pie,ori_religiosa,pago_matricula,pago_mensual
0,2014,1,LICEO POLITECNICO ARICA,15,15101,ARICA,2,1,0,-18.487274,-70.294739,1,2,1,1


In [24]:
pd.api.types.infer_dtype(ee_df["longitud"])

'mixed'

In [25]:
ee_df["longitud"] = ee_df["longitud"].apply(convert_to_float)
ee_df["latitud"] = ee_df["latitud"].apply(convert_to_float)

In [26]:
ee_df["longitud"] = pd.to_numeric(ee_df["longitud"])
ee_df["latitud"] = pd.to_numeric(ee_df["latitud"])

In [27]:
ee_df["latitud"].min()

np.float64(-331315662.0)

In [28]:
ee_df

,agno,rbd,nom_rbd,cod_reg_rbd,cod_com_rbd,nom_com_rbd,cod_depe,cod_depe2,rural_rbd,latitud,longitud,convenio_pie,ori_religiosa,pago_matricula,pago_mensual
0,2014,1,LICEO POLITECNICO ARICA,15,15101,ARICA,2,1,0,-18.487274,-70.294739,1,2,1,1
1,2014,2,PARVULARIO LAS ESPIGUITAS,15,15101,ARICA,2,1,0,-18.469862,-70.302757,1,2,1,1
2,2014,3,ESC. PEDRO VICENTE GUTIERREZ TORRES,15,15101,ARICA,2,1,1,-18.503580,-70.260826,0,2,1,1
3,2014,4,LICEO OCTAVIO PALMA PEREZ,15,15101,ARICA,2,1,0,-18.474243,-70.308266,1,1,1,1
4,2014,5,JOVINA NARANJO FERNANDEZ,15,15101,ARICA,2,1,0,-18.476904,-70.313454,1,7,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
158420,2023,42195,SALA CUNA Y JARDIN INFANTIL ARBOLITO,13,13130,SAN MIGUEL,4,3,0,NaN,NaN,0,9,SIN INFORMACION,SIN INFORMACION
158421,2023,42196,SALA CUNA QUITRAL MONTESSORI,13,13302,LAMPA,4,3,0,NaN,NaN,0,9,SIN INFORMACION,SIN INFORMACION
158422,2023,42197,SALA CUNA NIDO ALEXANDER MONTESSORI,13,13114,LAS CONDES,4,3,0,NaN,NaN,0,9,SIN INFORMACION,SIN INFORMACION
158423,2023,42198,COLEGIO DE ADULTOS AMANECER,13,13124,PUDAHUEL,3,2,0,NaN,NaN,0,9,SIN INFORMACION,SIN INFORMACION


In [29]:
%pip install pyarrow

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [30]:
ee_df.pago_matricula.value_counts()

pago_matricula
GRATUITO              100735
SIN INFORMACION        29764
1                       9722
$1.000 A $10.000        5990
MAS DE $100.000         3860
SIN INFORMACIÃN        3556
2                       1268
$50.001 A $100.000       918
$25.001 A $50.000        688
$10.001 A $25.000        447
0                        424
MÃS DE $100.000         406
6                        389
4                         96
5                         86
3                         76
Name: count, dtype: int64

In [31]:
def convert_pago_matricula(value):
    if pd.isna(value):
        return pd.NA
    if isinstance(value, (float, int)):
        return int(value)
    if isinstance(value, str):
        match value.strip():
            case "SIN INFORMACIÃN":
                return int(0)
            case "SIN INFORMACION":
                return int(0)
            case "GRATUITO":
                return int(1)
            case "$1.000 A $10.000":
                return int(2)
            case "$10.001 A $25.000":
                return int(3)
            case "$25.001 A $50.000":
                return int(4)
            case "$50.001 A $100.000":
                return int(5)
            case "MAS DE $100.000":
                return int(6)
            case "MÃS DE $100.000":
                return int(6)
    return pd.NA


In [32]:
ee_df.pago_matricula = ee_df.pago_matricula.apply(convert_pago_matricula)

In [33]:
ee_df.pago_mensual.value_counts()

pago_mensual
GRATUITO              93816
SIN INFORMACION       30397
1                      8746
$25.001 A $50.000      5057
MAS DE $100.000        4415
$50.001 A $100.000     4330
SIN INFORMACIÃN       3675
$10.001 A $25.000      3061
$1.000 A $10.000       1184
3                       797
4                       716
0                       507
2                       470
MÃS DE $100.000        429
5                       418
6                       407
Name: count, dtype: int64

In [34]:
ee_df.pago_mensual = ee_df.pago_mensual.apply(convert_pago_matricula)

In [35]:
ee_df.to_csv("../data/processed/establecimientos.csv")

In [36]:
ee_df.to_parquet("../data/processed/establecimientos.parquet")

## Sep Establecimientos

In [37]:
sep_ee_dfs = csv_loader(Path(column_path_dict["ee_sep"][0]))

print_columns(sep_ee_dfs)

Column name occurrences:
'AGNO' -> **9**
'RBD' -> **10**
'DGV_RBD' -> **10**
'NOM_RBD' -> **10**
'COD_REG_RBD' -> **10**
'COD_PRO_RBD' -> **10**
'COD_COM_RBD' -> **10**
'NOM_COM_RBD' -> **10**
'COD_DEPE' -> **10**
'COD_DEPE2' -> **10**
'RURAL_RBD' -> **10**
'CLASIFICACION_SEP' -> **10**
'N_PRIO' -> **10**
'N_BEN' -> **10**
'COD_DEPROV_RBD' -> **9**
'NOM_DEPROV_RBD' -> **9**
'CONVENIO_SEP' -> **9**
'AÑO_INGRESO_SEP' -> **9**
'ESTADO_ESTAB' -> **8**
'EE_GRATUITO' -> **8**
'N_PRIO_BEN' -> **8**
'N_PREF' -> **8**
'N_PREF_BEN' -> **8**
'﻿AGNO' -> **1**


In [38]:
for df in sep_ee_dfs:
    df.rename(columns={column:column.casefold().strip() for column in df.columns}, inplace=True)
    
for df in sep_ee_dfs:
    df.rename(columns={"\ufeffagno":"agno"}, inplace=True)
    
for df in sep_ee_dfs:
    df.rename(columns={"ïagno":"agno"}, inplace=True)

In [39]:
print_columns(sep_ee_dfs)

Column name occurrences:
'agno' -> **10**
'rbd' -> **10**
'dgv_rbd' -> **10**
'nom_rbd' -> **10**
'cod_reg_rbd' -> **10**
'cod_pro_rbd' -> **10**
'cod_com_rbd' -> **10**
'nom_com_rbd' -> **10**
'cod_depe' -> **10**
'cod_depe2' -> **10**
'rural_rbd' -> **10**
'clasificacion_sep' -> **10**
'n_prio' -> **10**
'n_ben' -> **10**
'cod_deprov_rbd' -> **9**
'nom_deprov_rbd' -> **9**
'convenio_sep' -> **9**
'año_ingreso_sep' -> **9**
'estado_estab' -> **8**
'ee_gratuito' -> **8**
'n_prio_ben' -> **8**
'n_pref' -> **8**
'n_pref_ben' -> **8**


In [40]:
sep_ee_df = pd.concat(sep_ee_dfs, axis=0, ignore_index=True)

In [41]:
columns_to_drop_sep_ee = [item for item in sep_ee_df.columns if item not in column_path_dict["ee_sep"][1]]

In [42]:
columns_to_drop_sep_ee

['dgv_rbd',
 'nom_rbd',
 'cod_reg_rbd',
 'cod_pro_rbd',
 'cod_com_rbd',
 'nom_com_rbd',
 'cod_depe',
 'cod_depe2',
 'rural_rbd',
 'cod_deprov_rbd',
 'nom_deprov_rbd',
 'año_ingreso_sep',
 'estado_estab',
 'ee_gratuito']

In [43]:
sep_ee_df.drop(columns=columns_to_drop_sep_ee, inplace=True)

In [44]:
sep_ee_df = sep_ee_df.replace(r'^\s*$', pd.NA, regex=True)

In [45]:
sep_ee_df.to_csv("../data/processed/sep_establecimientos.csv")

In [46]:
sep_ee_df.to_parquet("../data/processed/sep_establecimientos.parquet")

## Matriculas

In [47]:
matri_dfs = csv_loader(Path(column_path_dict["ee_matricula"][0]))

print_columns(matri_dfs)

Column name occurrences:
'ïagno' -> **3**
'rbd' -> **3**
'dgv_rbd' -> **3**
'nom_rbd' -> **3**
'cod_depe' -> **3**
'cod_depe2' -> **3**
'rural_rbd' -> **3**
'cod_reg_rbd' -> **3**
'cod_pro_rbd' -> **3**
'cod_com_rbd' -> **3**
'nom_com_rbd' -> **3**
'mat_hom_1' -> **3**
'mat_muj_1' -> **3**
'mat_ens_1' -> **3**
'mat_si_2' -> **3**
'mat_hom_2' -> **3**
'mat_muj_2' -> **3**
'mat_ens_2' -> **3**
'mat_hom_3' -> **3**
'mat_muj_3' -> **3**
'mat_ens_3' -> **3**
'mat_hom_4' -> **3**
'mat_muj_4' -> **3**
'mat_ens_4' -> **3**
'mat_hom_5' -> **3**
'mat_muj_5' -> **3**
'mat_ens_5' -> **3**
'mat_hom_6' -> **3**
'mat_muj_6' -> **3**
'mat_ens_6' -> **3**
'mat_hom_7' -> **3**
'mat_muj_7' -> **3**
'mat_ens_7' -> **3**
'mat_hom_8' -> **3**
'mat_muj_8' -> **3**
'mat_ens_8' -> **3**
'mat_hom_tot' -> **3**
'mat_muj_tot' -> **3**
'mat_si_tot' -> **3**
'mat_total' -> **3**
'cur_sim_01' -> **3**
'cur_sim_02' -> **3**
'cur_sim_03' -> **3**
'cur_sim_04' -> **3**
'cur_sim_05' -> **3**
'cur_sim_06' -> **3**
'cur_s

In [48]:
for df in matri_dfs:
    df.rename(columns={column:column.casefold().strip() for column in df.columns}, inplace=True)
    
for df in matri_dfs:
    df.rename(columns={"\ufeffagno":"agno"}, inplace=True)
    
for df in matri_dfs:
    df.rename(columns={"ïagno":"agno"}, inplace=True)

In [49]:
print_columns(matri_dfs)

Column name occurrences:
'agno' -> **10**
'rbd' -> **10**
'dgv_rbd' -> **10**
'nom_rbd' -> **10**
'cod_depe' -> **10**
'cod_depe2' -> **10**
'rural_rbd' -> **10**
'cod_reg_rbd' -> **10**
'cod_pro_rbd' -> **10**
'cod_com_rbd' -> **10**
'nom_com_rbd' -> **10**
'mat_hom_1' -> **10**
'mat_muj_1' -> **10**
'mat_ens_1' -> **10**
'mat_si_2' -> **10**
'mat_hom_2' -> **10**
'mat_muj_2' -> **10**
'mat_ens_2' -> **10**
'mat_hom_3' -> **10**
'mat_muj_3' -> **10**
'mat_ens_3' -> **10**
'mat_hom_4' -> **10**
'mat_muj_4' -> **10**
'mat_ens_4' -> **10**
'mat_hom_5' -> **10**
'mat_muj_5' -> **10**
'mat_ens_5' -> **10**
'mat_hom_6' -> **10**
'mat_muj_6' -> **10**
'mat_ens_6' -> **10**
'mat_hom_7' -> **10**
'mat_muj_7' -> **10**
'mat_ens_7' -> **10**
'mat_hom_8' -> **10**
'mat_muj_8' -> **10**
'mat_ens_8' -> **10**
'mat_hom_tot' -> **10**
'mat_muj_tot' -> **10**
'mat_si_tot' -> **10**
'mat_total' -> **10**
'cur_sim_01' -> **10**
'cur_sim_02' -> **10**
'cur_sim_03' -> **10**
'cur_sim_04' -> **10**
'cur_si

In [50]:
matri_df = pd.concat(matri_dfs, axis=0, ignore_index=True)

In [51]:
columns_to_drop_matri = [item for item in matri_df.columns if item not in column_path_dict["ee_matricula"][1]]

In [52]:
columns_to_drop_matri

['dgv_rbd',
 'cod_depe',
 'cod_depe2',
 'rural_rbd',
 'cod_reg_rbd',
 'cod_pro_rbd',
 'cod_com_rbd',
 'nom_com_rbd',
 'mat_ens_1',
 'mat_si_2',
 'mat_ens_2',
 'mat_ens_3',
 'mat_ens_4',
 'mat_ens_5',
 'mat_ens_6',
 'mat_ens_7',
 'mat_ens_8',
 'mat_si_tot',
 'cur_sim_01',
 'cur_sim_02',
 'cur_sim_03',
 'cur_sim_04',
 'cur_sim_05',
 'cur_sim_06',
 'cur_sim_07',
 'cur_sim_08',
 'cur_sim_tot',
 'cur_com_01',
 'cur_com_02',
 'cur_com_tot',
 'mat_hom_gd',
 'mat_muj_gd',
 'mat_gd_total',
 'cod_deprov_rbd',
 'nom_deprov_rbd',
 'estado_estab',
 'cur_comb_01',
 'cur_comb_02',
 'cur_comb_tot',
 'mat_si_1',
 'mat_si_4',
 'mat_si_8',
 'mat_si_3',
 'mat_si_5',
 'nom_reg_rbd_a',
 'unnamed: 0']

In [53]:
matri_df.drop(columns=columns_to_drop_matri, inplace=True)

In [54]:
matri_df = matri_df.replace(r'^\s*$', pd.NA, regex=True)

In [55]:
matri_df.to_csv("../data/processed/matriculas.csv")

In [56]:
matri_df.to_parquet("../data/processed/matriculas.parquet")